# Mechanizmy pozornosti a transformery

Jednou z hlavných nevýhod rekurentných sietí je, že všetky slová v sekvencii majú rovnaký vplyv na výsledok. To spôsobuje suboptimálny výkon pri štandardných modeloch LSTM encoder-decoder pre úlohy sekvencie na sekvenciu, ako je rozpoznávanie pomenovaných entít alebo strojový preklad. V skutočnosti majú konkrétne slová v vstupnej sekvencii často väčší vplyv na výstupy ako ostatné.

Zvážte model sekvencie na sekvenciu, napríklad strojový preklad. Ten je implementovaný pomocou dvoch rekurentných sietí, kde jedna sieť (**encoder**) zhrnie vstupnú sekvenciu do skrytého stavu a druhá (**decoder**) rozvinie tento skrytý stav do preloženého výsledku. Problém s týmto prístupom je, že konečný stav siete má problém zapamätať si začiatok vety, čo vedie k nízkej kvalite modelu pri dlhých vetách.

**Mechanizmy pozornosti** poskytujú spôsob, ako vážiť kontextuálny vplyv každého vstupného vektora na každú predikciu výstupu RNN. Implementuje sa to vytvorením skratiek medzi medzistavmi vstupnej RNN a výstupnej RNN. Týmto spôsobom, pri generovaní výstupného symbolu $y_t$, zohľadníme všetky skryté stavy vstupu $h_i$, s rôznymi váhovými koeficientmi $\alpha_{t,i}$.

![Obrázok zobrazujúci model encoder/decoder s vrstvou aditívnej pozornosti](../../../../../translated_images/sk/encoder-decoder-attention.7a726296894fb567.webp)
*Model encoder-decoder s mechanizmom aditívnej pozornosti podľa [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), citované z [tohto blogového príspevku](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Matica pozornosti $\{\alpha_{i,j}\}$ predstavuje mieru, do akej určité vstupné slová ovplyvňujú generovanie daného slova vo výstupnej sekvencii. Nižšie je príklad takejto matice:

![Obrázok zobrazujúci vzorové zarovnanie nájdené RNNsearch-50, prevzaté z Bahdanau - arviz.org](../../../../../translated_images/sk/bahdanau-fig3.09ba2d37f202a6af.webp)

*Obrázok prevzatý z [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Obr.3)*

Mechanizmy pozornosti sú zodpovedné za veľkú časť súčasného alebo takmer súčasného stavu umenia v spracovaní prirodzeného jazyka. Pridanie pozornosti však výrazne zvyšuje počet parametrov modelu, čo viedlo k problémom so škálovaním RNN. Kľúčovým obmedzením škálovania RNN je, že rekurentná povaha modelov sťažuje dávkovanie a paralelizáciu tréningu. V RNN musí byť každý prvok sekvencie spracovaný v sekvenčnom poradí, čo znamená, že ho nemožno ľahko paralelizovať.

Prijatie mechanizmov pozornosti v kombinácii s týmto obmedzením viedlo k vytvoreniu dnes už špičkových modelov Transformer, ktoré poznáme a používame, od BERT po OpenGPT3.

## Modely Transformer

Namiesto prenášania kontextu každej predchádzajúcej predikcie do ďalšieho kroku hodnotenia používajú **modely Transformer** **pozíciové kódovania** a **pozornosť**, aby zachytili kontext daného vstupu v rámci poskytnutého okna textu. Nižšie uvedený obrázok ukazuje, ako pozíciové kódovania s pozornosťou dokážu zachytiť kontext v rámci daného okna.

![Animovaný GIF zobrazujúci, ako sa hodnotenia vykonávajú v modeloch Transformer.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Keďže každá vstupná pozícia je mapovaná nezávisle na každú výstupnú pozíciu, transformery dokážu lepšie paralelizovať ako RNN, čo umožňuje oveľa väčšie a expresívnejšie jazykové modely. Každá hlava pozornosti môže byť použitá na učenie rôznych vzťahov medzi slovami, čo zlepšuje následné úlohy spracovania prirodzeného jazyka.

## Vytvorenie jednoduchého modelu Transformer

Keras neobsahuje zabudovanú vrstvu Transformer, ale môžeme si ju vytvoriť sami. Ako predtým, zameriame sa na klasifikáciu textu z datasetu AG News, ale stojí za zmienku, že modely Transformer dosahujú najlepšie výsledky pri náročnejších úlohách NLP.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Nové vrstvy v Keras by mali byť podtriedou triedy `Layer` a implementovať metódu `call`. Začnime s vrstvou **Positional Embedding**. Použijeme [niektorý kód z oficiálnej dokumentácie Keras](https://keras.io/examples/nlp/text_classification_with_transformer/). Predpokladáme, že všetky vstupné sekvencie doplníme na dĺžku `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Táto vrstva pozostáva z dvoch vrstiev `Embedding`: na vkladanie tokenov (spôsobom, o ktorom sme už hovorili) a na vkladanie pozícií tokenov. Pozície tokenov sa vytvárajú ako postupnosť prirodzených čísel od 0 po `maxlen` pomocou `tf.range`, a potom sa prechádzajú cez vrstvu vkladania. Dva výsledné vektorové vklady sa následne sčítajú, čím sa vytvorí pozične vložená reprezentácia vstupu s tvarom `maxlen`$\times$`embed_dim`.

Teraz implementujme blok transformátora. Bude prijímať výstup predtým definovanej vrstvy vkladania:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Transformer aplikuje `MultiHeadAttention` na vstup s pozičným kódovaním, aby vytvoril vektor pozornosti s rozmermi `maxlen`$\times$`embed_dim`, ktorý sa následne zmieša so vstupom a normalizuje pomocou `LayerNormalization`.

> **Poznámka**: `LayerNormalization` je podobná `BatchNormalization`, o ktorej sme hovorili v časti *Počítačové videnie* tohto vzdelávacieho kurzu, ale normalizuje výstupy predchádzajúcej vrstvy pre každú vzorku tréningu nezávisle, aby ich priviedla do rozsahu [-1..1].

Výstup tejto vrstvy sa potom prechádza cez sieť `Dense` (v našom prípade - dvojvrstvový perceptron) a výsledok sa pridáva k finálnemu výstupu (ktorý sa opäť podrobuje normalizácii).

Teraz sme pripravení definovať kompletný model transformera:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## BERT Transformer Modely

**BERT** (Bidirectional Encoder Representations from Transformers) je veľmi veľká viacvrstvová transformerová sieť s 12 vrstvami pre *BERT-base* a 24 vrstvami pre *BERT-large*. Model je najskôr predtrénovaný na veľkom korpuse textových dát (WikiPedia + knihy) pomocou nesupervidovaného učenia (predpovedanie maskovaných slov vo vete). Počas predtrénovania model absorbuje významnú úroveň porozumenia jazyka, ktorú je možné následne využiť s inými datasetmi pomocou jemného doladenia. Tento proces sa nazýva **transferové učenie**.

![obrázok z http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/sk/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Existuje mnoho variácií transformerových architektúr vrátane BERT, DistilBERT, BigBird, OpenGPT3 a ďalších, ktoré je možné jemne doladiť.

Pozrime sa, ako môžeme použiť predtrénovaný model BERT na riešenie nášho tradičného problému klasifikácie sekvencií. Požičajme si myšlienku a časť kódu z [oficiálnej dokumentácie](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

Na načítanie predtrénovaných modelov použijeme **Tensorflow hub**. Najskôr načítajme BERT-špecifický vektorizér:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Je dôležité, aby ste použili rovnaký vektorizačný nástroj, aký bol použitý pri pôvodnom tréningu siete. BERT vektorizačný nástroj vracia tri komponenty:
* `input_word_ids`, čo je sekvencia čísel tokenov pre vstupnú vetu
* `input_mask`, ktorá ukazuje, ktorá časť sekvencie obsahuje skutočný vstup a ktorá je výplňou. Je podobná maske vytvorenej vrstvou `Masking`
* `input_type_ids` sa používa pri úlohách jazykového modelovania a umožňuje špecifikovať dve vstupné vety v jednej sekvencii.

Potom môžeme inicializovať BERT extraktor vlastností:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Takže, BERT vrstva vracia niekoľko užitočných výsledkov:
* `pooled_output` je výsledkom spriemerovania všetkých tokenov v sekvencii. Môžete ho vnímať ako inteligentné sémantické zapuzdrenie celej siete. Je ekvivalentný výstupu vrstvy `GlobalAveragePooling1D` v našom predchádzajúcom modeli.
* `sequence_output` je výstup poslednej transformerovej vrstvy (zodpovedá výstupu `TransformerBlock` v našom vyššie uvedenom modeli).
* `encoder_outputs` sú výstupy všetkých transformerových vrstiev. Keďže sme načítali 4-vrstvový BERT model (ako pravdepodobne tušíte z názvu, ktorý obsahuje `4_H`), obsahuje 4 tenzory. Posledný z nich je rovnaký ako `sequence_output`.

Teraz definujeme end-to-end klasifikačný model. Použijeme *funkcionálnu definíciu modelu*, kde definujeme vstup modelu a potom poskytneme sériu výrazov na výpočet jeho výstupu. Tiež nastavíme váhy BERT modelu ako netrénovateľné a budeme trénovať iba finálny klasifikátor:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Napriek tomu, že je tu len málo trénovateľných parametrov, proces je pomerne pomalý, pretože extraktor vlastností BERT je výpočtovo náročný. Zdá sa, že sme nedosiahli rozumnú presnosť, buď kvôli nedostatku tréningu, alebo nedostatku parametrov modelu.

Skúsme odomknúť váhy BERT a trénovať ich tiež. To si vyžaduje veľmi malú rýchlosť učenia a tiež opatrnejšiu stratégiu tréningu s **warmup**, použitím optimalizátora **AdamW**. Na vytvorenie optimalizátora použijeme balík `tf-models-official`:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Ako môžete vidieť, tréning prebieha pomerne pomaly - ale možno budete chcieť experimentovať a trénovať model niekoľko epoch (5-10) a zistiť, či dosiahnete lepší výsledok v porovnaní s prístupmi, ktoré sme použili predtým.

## Knižnica Huggingface Transformers

Ďalším veľmi bežným (a o niečo jednoduchším) spôsobom, ako používať modely Transformer, je [balík HuggingFace](https://github.com/huggingface/), ktorý poskytuje jednoduché stavebné bloky pre rôzne NLP úlohy. Je dostupný pre Tensorflow aj PyTorch, ďalší veľmi populárny rámec pre neurónové siete.

> **Poznámka**: Ak vás nezaujíma, ako funguje knižnica Transformers - môžete preskočiť na koniec tohto notebooku, pretože neuvidíte nič podstatne odlišné od toho, čo sme robili vyššie. Budeme opakovať rovnaké kroky tréningu modelu BERT pomocou inej knižnice a podstatne väčšieho modelu. Proces preto zahŕňa pomerne dlhý tréning, takže možno budete chcieť len prejsť kódom.

Pozrime sa, ako môžeme náš problém vyriešiť pomocou [Huggingface Transformers](http://huggingface.co).


Prvá vec, ktorú musíme urobiť, je vybrať model, ktorý budeme používať. Okrem niektorých vstavaných modelov obsahuje Huggingface [online úložisko modelov](https://huggingface.co/models), kde môžete nájsť oveľa viac predtrénovaných modelov od komunity. Všetky tieto modely je možné načítať a používať len poskytnutím názvu modelu. Všetky potrebné binárne súbory pre model sa automaticky stiahnu.

V určitých prípadoch budete potrebovať načítať svoje vlastné modely, v takom prípade môžete špecifikovať adresár, ktorý obsahuje všetky relevantné súbory, vrátane parametrov pre tokenizer, súboru `config.json` s parametrami modelu, binárnych váh atď.

Z názvu modelu môžeme inicializovať model aj tokenizer. Začnime s tokenizerom:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Objekt `tokenizer` obsahuje funkciu `encode`, ktorú je možné priamo použiť na kódovanie textu:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Môžeme tiež použiť tokenizér na zakódovanie sekvencie spôsobom vhodným na odovzdanie modelu, t.j. vrátane polí `token_ids`, `input_mask` atď. Môžeme tiež špecifikovať, že chceme tenzory Tensorflow poskytnutím argumentu `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

V našom prípade budeme používať predtrénovaný BERT model nazývaný `bert-base-uncased`. *Uncased* znamená, že model je necitlivý na veľkosť písmen.

Pri trénovaní modelu musíme poskytnúť tokenizovanú sekvenciu ako vstup, a preto navrhneme dátový spracovateľský pipeline. Keďže `tokenizer.encode` je Python funkcia, použijeme rovnaký prístup ako v poslednej jednotke, kde ju voláme pomocou `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Teraz môžeme načítať skutočný model pomocou balíka `BertForSequenceClassification`. To zabezpečuje, že náš model už má požadovanú architektúru pre klasifikáciu, vrátane konečného klasifikátora. Uvidíte varovnú správu, ktorá uvádza, že váhy konečného klasifikátora nie sú inicializované a model by vyžadoval predtréning - to je úplne v poriadku, pretože presne to sa chystáme urobiť!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Ako môžete vidieť z `summary()`, model obsahuje takmer 110 miliónov parametrov! Predpokladáme, že ak chceme jednoduchú klasifikačnú úlohu na relatívne malom datasete, nechceme trénovať základnú vrstvu BERT:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Teraz sme pripravení začať tréning!

> **Poznámka**: Tréning plnohodnotného BERT modelu môže byť veľmi časovo náročný! Preto ho budeme trénovať iba na prvých 32 dávkach. Toto slúži len na ukážku, ako je nastavený tréning modelu. Ak máte záujem vyskúšať plnohodnotný tréning - jednoducho odstráňte parametre `steps_per_epoch` a `validation_steps` a pripravte sa čakať!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Ak zvýšite počet iterácií a počkáte dostatočne dlho, a trénujete počas viacerých epoch, môžete očakávať, že klasifikácia pomocou BERT nám poskytne najlepšiu presnosť! Je to preto, že BERT už veľmi dobre rozumie štruktúre jazyka, a my potrebujeme len doladiť záverečný klasifikátor. Avšak, keďže BERT je veľký model, celý proces trénovania trvá dlho a vyžaduje značný výpočtový výkon! (GPU, a ideálne viac ako jednu).

> **Note:** V našom príklade sme používali jeden z najmenších predtrénovaných modelov BERT. Existujú väčšie modely, ktoré pravdepodobne prinesú lepšie výsledky.


## Hlavné body

V tejto jednotke sme sa oboznámili s najnovšími modelovými architektúrami založenými na **transformeroch**. Aplikovali sme ich na našu úlohu klasifikácie textu, ale podobne môžu byť BERT modely použité na extrakciu entít, odpovedanie na otázky a iné NLP úlohy.

Transformačné modely predstavujú súčasný špičkový stav v NLP a vo väčšine prípadov by mali byť prvým riešením, s ktorým začnete experimentovať pri implementácii vlastných NLP riešení. Avšak pochopenie základných princípov rekurentných neurónových sietí, o ktorých sme hovorili v tomto module, je mimoriadne dôležité, ak chcete vytvárať pokročilé neurónové modely.



---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby AI prekladu [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, prosím, berte na vedomie, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nie sme zodpovední za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
